In [8]:
import json
import sys
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup as Soup


univ_forum_map = pd.read_csv("Links.csv",header=0)

def fetch_answers(my_url):
    req = requests.get(my_url)
    print("Reading URL......"+str(my_url))
    page_soup = Soup(req.content, "html.parser")
    main_box = page_soup.findAll("script", {"type": "application/ld+json"})[0].text

    data = json.loads(main_box)
    answers = []
    suggested_links = []
    try:
        answers += [x["text"] for x in data["mainEntity"]["acceptedAnswer"]  if 'parentItem' not in x.keys()]
        suggested_links += [x["parentItem"]["url"] for x in data["mainEntity"]["acceptedAnswer"]  if 'parentItem' in x.keys()]
    except:
        pass
        answers += [x["text"] for x in data["mainEntity"]["suggestedAnswer"]  if 'parentItem' not in x.keys()]
        suggested_links += [x["parentItem"]["url"] for x in data["mainEntity"]["suggestedAnswer"]  if 'parentItem' in x.keys()]

    return answers,suggested_links

def concat_answers(reviews, answers,university,url):
  data = []
  parts = url.split("https://www.quora.com/")
  question = parts[1] if len(parts)>1 else None
  for answer in answers:
    data.append({'University': university, 'Comment': answer, 'Question': question})
  df = pd.DataFrame(data)
  reviews= pd.concat([reviews, df], ignore_index=True)
  return reviews


reviews = pd.DataFrame({})
for index, row in univ_forum_map.iterrows():
  answers,suggested_links = fetch_answers(row['Link']) 
  reviews= concat_answers(reviews, answers,row['University'],row['Link'])

  for suggested_link in suggested_links:
    suggested_answers,tmp = fetch_answers(suggested_link)
    reviews= concat_answers(reviews, suggested_answers,row['University'],suggested_link)

reviews.drop_duplicates()
reviews.to_csv("./Files/total_reviews.csv")




Reading URL......https://www.quora.com/How-is-a-Wharton-MBA-different-from-the-Booth-MBA-program
Reading URL......https://www.quora.com/Why-is-Booth-perceived-as-a-less-renowned-subset-of-Wharton
Reading URL......https://www.quora.com/Is-the-Wharton-MBA-worth-it
Reading URL......https://www.quora.com/Why-is-Booth-perceived-to-be-inferior-to-Wharton-and-Stanford
Reading URL......https://www.quora.com/Which-is-better-a-Wharton-MBA-or-a-Harvard-MBA
Reading URL......https://www.quora.com/Why-is-University-of-Chicago-Booth-School-of-Business-rated-so-highly-these-days-with-more-and-more-aspirants-yearning-to-get-an-MBA-from-there
Reading URL......https://www.quora.com/Why-would-anyone-go-to-Wharton-for-an-MBA
Reading URL......https://www.quora.com/Is-it-worth-it-to-attend-an-MBA-program-that-is-not-HBS-Stanford-GSB-Wharton-or-Chicago-Booth
Reading URL......https://www.quora.com/Why-is-Booth-perceived-to-be-inferior-to-Wharton-and-Stanford
Reading URL......https://www.quora.com/How-was-your-